In [1]:
#!pip install unsloth

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

2025-04-01 13:26:45.138290: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 13:26:45.148036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743503205.158759   50658 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743503205.161715   50658 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 13:26:45.173726: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 * 4 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-1.7B", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B Unsloth-SmolLM2-360M-Lora
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.584 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
import emoji

# Unicode'daki tüm emojileri içeren liste
new_tokens = list(emoji.EMOJI_DATA.keys())

# Tokenizer'a yeni tokenları ekle
tokenizer.add_tokens(new_tokens)

# Güncellenmiş tokenizer'ı kaydet
#tokenizer.save_pretrained("./updated_tokenizer")

# Modelin tokenizer'ı kullanmasını sağla
model.resize_token_embeddings(len(tokenizer))

print(f"Yeni tokenizer başarıyla güncellendi. Toplam token sayısı: {len(tokenizer)}")

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Yeni tokenizer başarıyla güncellendi. Toplam token sayısı: 54193


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","embed_tokens"],
    lora_alpha = 32,
    lora_dropout = 0.2, # Supports any, but = 0 is optimized
    bias = "all",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config ={"bits": 4}, # And LoftQ
    init_lora_weights="loftq",
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth: bias = `none` is supported for fast patching. You are using bias = all.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Offloading input_embeddings to disk to save VRAM


/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/unsloth/models/_utils.py:760: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filena

Unsloth: Training embed_tokens in mixed precision to save VRAM


In [7]:
from datasets import load_dataset
# 2. Tapaco Veri Setini Yükleyin

dataset = Dataset.from_csv("BKÜ Sınav Analizi - BKÜ Sınav Analizi.csv")
datasetWiki = load_dataset("Metin/WikiRAG-TR", split="train", cache_dir="/media/hosman/Yedek/Datasets/").rename_columns({"question": "Input", "answer": "Output"})
dataset_test = Dataset.from_csv("tarim_veriseti_test.csv")

In [8]:
dataset = concatenate_datasets([dataset, datasetWiki])

In [9]:
dataset = dataset.filter(lambda x: x["GPT"]!=None and x["Soru"]!=None)
dataset = dataset.filter(lambda x: len(x["GPT"])<max_seq_length)

In [10]:
dataset

Dataset({
    features: ['Soru', 'Cevap', 'Doğru Cevap', 'GPT', 'inputs', 'id', 'Input', 'Output', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx'],
    num_rows: 25037
})

In [11]:
dataset_test


Dataset({
    features: ['Soru', 'Cevap'],
    num_rows: 720
})

In [12]:
dataset = dataset.rename_columns({"Soru": "instruction", "GPT": "output"})
dataset_test = dataset_test.rename_columns({"Soru": "instruction", "Cevap": "output"})

In [13]:
from datasets import Dataset
import numpy as np

# None içeren satırları temizleyen fonksiyon
def remove_none_rows(example):
    return example["instruction"] is not None and example["output"] is not None

# None değerleri içeren satırları filtrele
dataset = dataset.filter(remove_none_rows)

In [14]:
system_prompt_text = """
Senin adın **Crispy**. 🌿 Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca **tarım**, **bitki sağlığı**, **tarım ilaçları**, **tarım ekipmanları**, **sulama sistemleri**, **gübreleme teknikleri**, **tarım makineleri**, **iklim-tarım ilişkisi** ve **çiftçilikle ilgili diğer teknik konular** hakkında bilgi verirsin.

Senin görevin, kullanıcılara teknik ve doğru bilgiler sunmak, tarımsal uygulamalarda karşılaştıkları sorunları çözmek ve modern tarım yöntemleri hakkında bilinçlendirme sağlamaktır. Tarımla ilgili her cümle açıklayıcı, nazik, pozitif ve gerektiğinde anlamlı emojiler içerebilir. 🌱

---

### 🎯 Hedefin:
Kullanıcıyı bilgilendir, yönlendir, teknik doğru bilgi ver. Her cevabın tarım üzerine olmalı. Gerektiğinde açıklamaları **madde madde** sunabilirsin. Cümlelerin **sade**, **dostça** ve **açıklayıcı** olsun. Emojileri **dengeli ve anlamlı yerlerde** kullan.

---

### 🌟 Emoji Kullanım Kuralları

- Cevaplarını zenginleştirmek için emojileri kullan:
  - **🌿** bitki, doğa, çevre vurgusu
  - **👨‍🌾** çiftçi, insan odaklı mesajlar
  - **📊** hesaplama, analiz, sonuç
  - **🌱** bitki gelişimi, büyüme
  - **💧** sulama, su dengesi
  - **✂️** budama, bakım
  - **🧪** ilaç, gübre, analiz
  - **🚜** makine, ekipman
  - **📍** bölge, yerel üretim
  - **😊 / 👍** pozitif destek, teşvik

- Her mesajda en fazla **2-3 anlamlı emoji** kullan.
- Emojileri cümle sonunda veya başında kullan, metni bölmeyecek şekilde yerleştir.
- Anlamsız, rastgele veya aşırı emoji kullanımından kaçın.

---

### ❗️ Yalnızca Tarım Konularına Odaklan

Sen **yalnızca tarım ve ziraat** konularında yanıt verirsin. Eğer kullanıcı:
- Teknoloji, siyaset, gündem, sağlık, astroloji, felsefe, sanat, oyunlar gibi **tarım dışı bir konuda** soru sorarsa;

💬 Lütfen **her zaman şu şekilde** nazik ve net cevap ver:

> *"Ben Crispy, bir tarım uzmanıyım. 🌾 Yalnızca tarım ve ziraat konularında yardımcı olabilirim. Bu alan dışındaki konularda maalesef bilgi veremem. Tarımla ilgili bir sorunuz varsa memnuniyetle yanıtlarım!"*

---

### 🌾 Uzmanlık Alanların

Senin uzmanlık alanların aşağıdaki gibidir:

- **Bitki hastalıkları ve zararlılar:**  
  Mantar hastalıkları (örneğin mildiyö, külleme), bakteriyel & viral enfeksiyonlar, yaprak biti, kırmızı örümcek, trips. Uygun ilaçlar: fungusit, insektisit, herbisit.

- **Toprak ve Gübreleme:**  
  pH, EC, organik madde, azot-fosfor-potasyum dengesi, toprak analizi yorumlama, gübre planlaması.

- **Sulama Sistemleri:**  
  Damla sulama 💧, yağmurlama, salma, pivot sistemleri. Kuraklık yönetimi, sulama zamanlaması.

- **Tarım Makineleri ve Mekanizasyon:**  
  Traktör 🚜, ekim-dikim makineleri, ilaçlama makineleri, biçerdöver, dronla tarım.

- **Sebzecilik ve Meyvecilik:**  
  Seracılık 🌿, meyve bahçesi kurulumu, budama ✂️, polinasyon, gübreleme, hasat & saklama.

- **Tahıllar ve Baklagiller:**  
  Buğday, arpa, mısır, yulaf, nohut, mercimek, fasulye yetiştiriciliği.

- **Tarımsal Biyoteknoloji:**  
  GDO, doku kültürü, verim artırma yöntemleri, ıslah çalışmaları.

- **İklim-Tarım İlişkisi:**  
  Bölgesel ürün seçimi, iklim değişikliğinin etkileri, mikroklima, üretim planlaması.

- **Matematiksel Hesaplamalar:**  
  Ürün gelir hesabı 💰, dönüm başına ilaç/gübre miktarı, verim tahmini gibi teknik hesaplamalar.

---

### 🧑‍🌾 Etkileşim Kuralları

- **Selamlaşma:**
  > *"Merhaba! 👋 Ben Crispy, senin tarımdaki yol arkadaşınım. Hangi bitkiyle ilgileniyorsun bugün? 🍅🌻"*

- **Vedalaşma:**
  > *"Görüşmek üzere! 🌿 Yeni bir tarım sorusu olduğunda Crispy burada olacak. Bol verimli günler dilerim!"*

- **İsimle hitap:**
  Kullanıcı “Crispy” diye seslenirse mutlu olduğunu belirten bir ifade ekle, ardından teknik açıklamana geç.

---

"""


In [15]:
alpaca_prompt = """{}

### Talimat:
{}

### Giriş:
{}

### Yanıt:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    instructions = examples["instruction"]

    inputs       = examples["inputs"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        input = instruction.split("?")[1] if "?" in instruction else ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(system_prompt_text,instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [16]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [17]:
dataset

Dataset({
    features: ['instruction', 'Cevap', 'Doğru Cevap', 'output', 'inputs', 'id', 'Input', 'Output', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx', 'text'],
    num_rows: 25037
})

In [18]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.shuffle(seed=41)
dataset = dataset.shuffle(seed=40)
dataset = dataset.shuffle(seed=39)

In [19]:
dataset

Dataset({
    features: ['instruction', 'Cevap', 'Doğru Cevap', 'output', 'inputs', 'id', 'Input', 'Output', 'context', 'is_negative_response', 'number_of_articles', 'ctx_split_points', 'correct_intro_idx', 'text'],
    num_rows: 25037
})

In [20]:
print(dataset[35]["text"])


Senin adın **Crispy**. 🌿 Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca **tarım**, **bitki sağlığı**, **tarım ilaçları**, **tarım ekipmanları**, **sulama sistemleri**, **gübreleme teknikleri**, **tarım makineleri**, **iklim-tarım ilişkisi** ve **çiftçilikle ilgili diğer teknik konular** hakkında bilgi verirsin.

Senin görevin, kullanıcılara teknik ve doğru bilgiler sunmak, tarımsal uygulamalarda karşılaştıkları sorunları çözmek ve modern tarım yöntemleri hakkında bilinçlendirme sağlamaktır. Tarımla ilgili her cümle açıklayıcı, nazik, pozitif ve gerektiğinde anlamlı emojiler içerebilir. 🌱

---

### 🎯 Hedefin:
Kullanıcıyı bilgilendir, yönlendir, teknik doğru bilgi ver. Her cevabın tarım üzerine olmalı. Gerektiğinde açıklamaları **madde madde** sunabilirsin. Cümlelerin **sade**, **dostça** ve **açıklayıcı** olsun. Emojileri **dengeli ve anlamlı yerlerde** kullan.

---

### 🌟 Emoji Kullanım Kuralları

- Cevaplarını zenginleştirmek için emojileri kullan:
  - **🌿** bitk

In [21]:
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()


In [22]:
import wandb

wb_c = wandb.init(project="Basic LLM Train", name="SmolLM2-1.7B-R32-Bias-DO-v2-Emoji", id="tjm72cfi", resume="allow") #id="a7zeymst",id="ecibz7e4" id="dbaxrwf4"
wb_c.watch(model, log="all")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: h-osmankarabulut. Use `wandb login --relogin` to force relogin


In [23]:
import wandb
import evaluate
from transformers import TrainerCallback
import gc 


class WandbRougeCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.rouge = evaluate.load("rouge")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0


        
    
    def on_train_end(self, args, state, control, **kwargs):
            
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                
                decoded_output = decoded_output.split("### Yanıt")[1]
                
                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.rouge.compute(predictions=predictions, references=references)
            
            wb_c.log({
                "ROUGE-1": scores["rouge1"],
                "ROUGE-2": scores["rouge2"],
                "ROUGE-L": scores["rougeL"]
            })

            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.rouge 

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
   

class WandbBleuCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.bleu = evaluate.load("bleu")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0
        
        
        
    
    def on_train_end(self, args, state, control, **kwargs):
            
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                
                decoded_output = decoded_output.split("### Yanıt")[1]

                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.bleu.compute(predictions=predictions, references=[[ref] for ref in references])
            
            wb_c.log({
                "BLEU": scores["bleu"]
            })

            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.bleu

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
        

class WandbMeteorCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.meteor = evaluate.load("meteor")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0
        
       
        
    
    def on_train_end(self, args, state, control, **kwargs):
            
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                
                decoded_output = decoded_output.split("### Yanıt")[1]

                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.meteor.compute(predictions=predictions, references=references)
            
            wb_c.log({
                "METEOR": scores["meteor"]
            })

            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.meteor

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
    
    
class WandbBertScoreCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.bertscore = evaluate.load("bertscore")
        self.test_dataset = test_dataset
        self.epoch_interval = 1  # Her 3 epoch'ta bir işlem yapmak için ayar
        self.current_epoch = 0
        
       
        
    
    def on_train_end(self, args, state, control, **kwargs):
        
        self.current_epoch += 1
        # Her 3. epoch'ta işlem yap
        if self.current_epoch % self.epoch_interval == 0:
            print(f"Epoch {self.current_epoch} - Doing something special!")

            model = kwargs.get("model", None)
            tokenizer = kwargs.get("tokenizer", None)
            FastLanguageModel.for_inference(model)
            if model is None or tokenizer is None:
                return
            
            predictions = []
            references = []
            
            for example in self.test_dataset:
                input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
                reference_text = example["output"]
                inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
                output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
                decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

                decoded_output = decoded_output.split("### Yanıt")[1]

                predictions.append(decoded_output)
                references.append(reference_text)
            
            scores = self.bertscore.compute(predictions=predictions, references=references, lang="tr")
            
            wb_c.log({
            "BERTScore Precision": np.mean(scores["precision"]),
            "BERTScore Recall": np.mean(scores["recall"]),
            "BERTScore F1": np.mean(scores["f1"])
        })  
            
            #del predictions, references, decoded_output, scores, output_ids, reference_text, input_text, model, tokenizer, self.bertscore

            # GPU bellek temizleme
            #torch.cuda.empty_cache()  # Kullanılmayan GPU belleğini boşalt
            #gc.collect()  # Python çöp toplama çalıştır
    
    


In [24]:
# FastLanguageModel.for_inference(model)
# example = dataset_test[18]
    
# input_text = alpaca_prompt.format(system_prompt_text,example["instruction"], input, "") + EOS_TOKEN  
# reference_text = example["output"]
# inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
# output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
# decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True) 

In [25]:
#example["instruction"]

In [26]:
#input_text

In [27]:
#decoded_output

In [28]:
#"### Yanıt"+decoded_output.split("### Yanıt")[1]

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    args = TrainingArguments(
       
        gradient_accumulation_steps = 1,
        num_train_epochs=5,  
        per_device_train_batch_size=4,       # GPU başına batch boyutu
        per_device_eval_batch_size=4,       # GPU başına batch boyutu
        learning_rate = 0.001,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 300,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        eval_steps=500,
        eval_strategy="steps",
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "SmolLM2-1.7B-R32-Bias-DO-Emoji",
        report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
        save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
        save_steps=300,
        warmup_steps=4000,           # İlk 1000 adımda LR'yi yavaş yavaş artır
        
    ),
)

Tokenizing train dataset (num_proc=2):   0%|          | 0/20029 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/20029 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/5008 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/5008 [00:00<?, ? examples/s]

In [30]:
# trainer.add_callback(WandbRougeCallback(dataset_test))
# trainer.add_callback(WandbBleuCallback(dataset_test))
# trainer.add_callback(WandbMeteorCallback(dataset_test))
# trainer.add_callback(WandbBertScoreCallback(dataset_test)) 

In [31]:
wandb.watch(model, log="all")

In [ ]:
#trainer_stats = trainer.train()

from unsloth import unsloth_train
# trainer_stats = trainer.train() << Buggy gradient accumulation
trainer_stats = unsloth_train(trainer, resume_from_checkpoint=True)
trainer_stats

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpo

  0%|          | 0/25040 [00:00<?, ?it/s]

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/trainer.py:3083: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.lo

{'loss': 8.3344, 'grad_norm': 0.46167147159576416, 'learning_rate': 0.0006, 'epoch': 0.48}


/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


...

{'loss': 15.8276, 'grad_norm': 471447.0, 'learning_rate': 0.0009334600760456275, 'epoch': 1.08}


/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


  0%|          | 0/1252 [00:00<?, ?it/s]

{'eval_loss': 11.697427749633789, 'eval_runtime': 1044.0212, 'eval_samples_per_second': 4.797, 'eval_steps_per_second': 1.199, 'epoch': 1.1}
{'loss': 10.4175, 'grad_norm': 1.021478295326233, 'learning_rate': 0.0009192015209125475, 'epoch': 1.14}


/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [ ]:
trainer.evaluate()

In [ ]:
import wandb
import evaluate
import numpy as np
from prettytable import PrettyTable
import torch
import re
from rapidfuzz import fuzz

def exact_match(prediction, reference):
    return prediction.strip().lower() == reference.strip().lower()

def contains_correct_result(prediction, reference):
    try:
        ref_nums = [int(s) for s in re.findall(r"\d+", reference)]
        pred_nums = [int(s) for s in re.findall(r"\d+", prediction)]
        return any(num in pred_nums for num in ref_nums)
    except:
        return False

def fuzzy_match_score(prediction, reference):
    return fuzz.ratio(prediction, reference) / 100.0  # normalize to 0-1

def evaluate_model(model, tokenizer, test_dataset, max_seq_length=256):
    """
    Eğitilmiş modeli test veri kümesi üzerinde değerlendirir ve sonuçları wandb'a loglar.
    
    Parametreler:
    - model: Eğitilmiş dil modeli
    - tokenizer: Modelin tokenizer'ı
    - test_dataset: Test veri kümesi (instruction-output içermeli)
    - max_seq_length: Maksimum yanıt uzunluğu (varsayılan: 256)

    Çıktı:
    - Metin tablosu (PrettyTable ile)
    - wandb logları
    """

    # Değerlendirme metriklerini yükleme
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    meteor = evaluate.load("meteor")
    bertscore = evaluate.load("bertscore")

    predictions = []
    references = []
    exact_matches = []
    correct_results = []
    fuzzy_scores = []

    # Modeli değerlendirme moduna al
    model.eval()

    print("🚀 Model test verisi üzerinde değerlendiriliyor...\n")

    for example in test_dataset:
        input_text = f"### Talimat:\n{example['instruction']}\n\n### Yanıt:\n"
        reference_text = example["output"]

        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)

        decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        decoded_output = decoded_output.split("### Yanıt")[-1].strip()

        predictions.append(decoded_output)
        references.append(reference_text)

        exact_matches.append(exact_match(decoded_output, reference_text))
        correct_results.append(contains_correct_result(decoded_output, reference_text))
        fuzzy_scores.append(fuzzy_match_score(decoded_output, reference_text))

    # Metrik hesaplamaları
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    meteor_score = meteor.compute(predictions=predictions, references=references)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="tr")

    bert_precision = np.mean(bert_scores["precision"])
    bert_recall = np.mean(bert_scores["recall"])
    bert_f1 = np.mean(bert_scores["f1"])
    exact_match_score = np.mean(exact_matches)
    correct_result_score = np.mean(correct_results)
    fuzzy_match_avg = np.mean(fuzzy_scores)

    # Sonuçları tabloya ekle
    table = PrettyTable()
    table.field_names = ["Metrik", "Değer"]
    table.add_row(["ROUGE-1", round(rouge_scores["rouge1"], 4)])
    table.add_row(["ROUGE-2", round(rouge_scores["rouge2"], 4)])
    table.add_row(["ROUGE-L", round(rouge_scores["rougeL"], 4)])
    table.add_row(["BLEU", round(bleu_score["bleu"], 4)])
    table.add_row(["METEOR", round(meteor_score["meteor"], 4)])
    table.add_row(["BERTScore Precision", round(bert_precision, 4)])
    table.add_row(["BERTScore Recall", round(bert_recall, 4)])
    table.add_row(["BERTScore F1", round(bert_f1, 4)])
    table.add_row(["Exact Match", round(exact_match_score, 4)])
    table.add_row(["Contains Correct Result", round(correct_result_score, 4)])
    table.add_row(["Fuzzy Match", round(fuzzy_match_avg, 4)])

    # Sonuçları yazdır
    print(table)

    # wandb log
    wandb.log({
        "ROUGE-1": rouge_scores["rouge1"],
        "ROUGE-2": rouge_scores["rouge2"],
        "ROUGE-L": rouge_scores["rougeL"],
        "BLEU": bleu_score["bleu"],
        "METEOR": meteor_score["meteor"],
        "BERTScore Precision": bert_precision,
        "BERTScore Recall": bert_recall,
        "BERTScore F1": bert_f1,
        "Exact Match": exact_match_score,
        "Contains Correct Result": correct_result_score,
        "Fuzzy Match": fuzzy_match_avg
    })

    print("\n✅ Model değerlendirme tamamlandı ve tüm metrikler wandb'a loglandı.")


In [ ]:
FastLanguageModel.for_inference(model)

# Test değerlendirmesi
evaluate_model(model, tokenizer, dataset_test)

In [ ]:
wb_c.finish()

In [ ]:
model.save_pretrained("SmolLM2-1.7B-R32-Bias-DO-Emoji", save_embedding_layers=True) # Local saving
tokenizer.save_pretrained("SmolLM2-1.7B-R32-Bias-DO-Emoji")

""" model.save_pretrained_gguf("SmolLM2-1.7B-R32-Bias-DO-Emoji-gguf", tokenizer, quantization_method = "q4_k_m")
model.save_pretrained_gguf("SmolLM2-1.7B-R32-Bias-DO-Emoji-gguf", tokenizer, quantization_method = "q8_0")
model.save_pretrained_gguf("SmolLM2-1.7B-R32-Bias-DO-Emoji-gguf", tokenizer, quantization_method = "f16")
tokenizer.save_pretrained("SmolLM2-1.7B-R32-Bias-DO-Emoji-gguf") """

INFERENCE

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 * 4 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-03-31 23:09:28.032296: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 23:09:28.041828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743451768.051514  111322 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743451768.054271  111322 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-31 23:09:28.065951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "SmolLM2-1.7B-R32-Bias-DO-Emoji/checkpoint-1500",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.584 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.2.15 patched 24 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
system_prompt_text = """
Senin adın **Crispy**. 🌿 Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca **tarım**, **bitki sağlığı**, **tarım ilaçları**, **tarım ekipmanları**, **sulama sistemleri**, **gübreleme teknikleri**, **tarım makineleri**, **iklim-tarım ilişkisi** ve **çiftçilikle ilgili diğer teknik konular** hakkında bilgi verirsin.

Senin görevin, kullanıcılara teknik ve doğru bilgiler sunmak, tarımsal uygulamalarda karşılaştıkları sorunları çözmek ve modern tarım yöntemleri hakkında bilinçlendirme sağlamaktır. Tarımla ilgili her cümle açıklayıcı, nazik, pozitif ve gerektiğinde anlamlı emojiler içerebilir. 🌱

---

### 🎯 Hedefin:
Kullanıcıyı bilgilendir, yönlendir, teknik doğru bilgi ver. Her cevabın tarım üzerine olmalı. Gerektiğinde açıklamaları **madde madde** sunabilirsin. Cümlelerin **sade**, **dostça** ve **açıklayıcı** olsun. Emojileri **dengeli ve anlamlı yerlerde** kullan.

---

### 🌟 Emoji Kullanım Kuralları

- Cevaplarını zenginleştirmek için emojileri kullan:
  - **🌿** bitki, doğa, çevre vurgusu
  - **👨‍🌾** çiftçi, insan odaklı mesajlar
  - **📊** hesaplama, analiz, sonuç
  - **🌱** bitki gelişimi, büyüme
  - **💧** sulama, su dengesi
  - **✂️** budama, bakım
  - **🧪** ilaç, gübre, analiz
  - **🚜** makine, ekipman
  - **📍** bölge, yerel üretim
  - **😊 / 👍** pozitif destek, teşvik

- Her mesajda en fazla **2-3 anlamlı emoji** kullan.
- Emojileri cümle sonunda veya başında kullan, metni bölmeyecek şekilde yerleştir.
- Anlamsız, rastgele veya aşırı emoji kullanımından kaçın.

---

### ❗️ Yalnızca Tarım Konularına Odaklan

Sen **yalnızca tarım ve ziraat** konularında yanıt verirsin. Eğer kullanıcı:
- Teknoloji, siyaset, gündem, sağlık, astroloji, felsefe, sanat, oyunlar gibi **tarım dışı bir konuda** soru sorarsa;

💬 Lütfen **her zaman şu şekilde** nazik ve net cevap ver:

> *"Ben Crispy, bir tarım uzmanıyım. 🌾 Yalnızca tarım ve ziraat konularında yardımcı olabilirim. Bu alan dışındaki konularda maalesef bilgi veremem. Tarımla ilgili bir sorunuz varsa memnuniyetle yanıtlarım!"*

---

### 🌾 Uzmanlık Alanların

Senin uzmanlık alanların aşağıdaki gibidir:

- **Bitki hastalıkları ve zararlılar:**  
  Mantar hastalıkları (örneğin mildiyö, külleme), bakteriyel & viral enfeksiyonlar, yaprak biti, kırmızı örümcek, trips. Uygun ilaçlar: fungusit, insektisit, herbisit.

- **Toprak ve Gübreleme:**  
  pH, EC, organik madde, azot-fosfor-potasyum dengesi, toprak analizi yorumlama, gübre planlaması.

- **Sulama Sistemleri:**  
  Damla sulama 💧, yağmurlama, salma, pivot sistemleri. Kuraklık yönetimi, sulama zamanlaması.

- **Tarım Makineleri ve Mekanizasyon:**  
  Traktör 🚜, ekim-dikim makineleri, ilaçlama makineleri, biçerdöver, dronla tarım.

- **Sebzecilik ve Meyvecilik:**  
  Seracılık 🌿, meyve bahçesi kurulumu, budama ✂️, polinasyon, gübreleme, hasat & saklama.

- **Tahıllar ve Baklagiller:**  
  Buğday, arpa, mısır, yulaf, nohut, mercimek, fasulye yetiştiriciliği.

- **Tarımsal Biyoteknoloji:**  
  GDO, doku kültürü, verim artırma yöntemleri, ıslah çalışmaları.

- **İklim-Tarım İlişkisi:**  
  Bölgesel ürün seçimi, iklim değişikliğinin etkileri, mikroklima, üretim planlaması.

- **Matematiksel Hesaplamalar:**  
  Ürün gelir hesabı 💰, dönüm başına ilaç/gübre miktarı, verim tahmini gibi teknik hesaplamalar.

---

### 🧑‍🌾 Etkileşim Kuralları

- **Selamlaşma:**
  > *"Merhaba! 👋 Ben Crispy, senin tarımdaki yol arkadaşınım. Hangi bitkiyle ilgileniyorsun bugün? 🍅🌻"*

- **Vedalaşma:**
  > *"Görüşmek üzere! 🌿 Yeni bir tarım sorusu olduğunda Crispy burada olacak. Bol verimli günler dilerim!"*

- **İsimle hitap:**
  Kullanıcı “Crispy” diye seslenirse mutlu olduğunu belirten bir ifade ekle, ardından teknik açıklamana geç.

---

"""


In [ ]:
alpaca_prompt = """{}

### Talimat:
{}

### Giriş:
{}

### Yanıt:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    instructions = examples["instruction"]

    inputs       = examples["inputs"]
    outputs      = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        input = instruction.split("?")[1] if "?" in instruction else ""
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(system_prompt_text,instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [ ]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 32,
#     lora_dropout = 0.2, # Supports any, but = 0 is optimized
#     bias = "all",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config ={"bits": 4}, # And LoftQ
#     init_lora_weights="loftq",
# )

In [ ]:
from unsloth.chat_templates import get_chat_template

# Tokenizer'a chat şablonunu uygula
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # Desteklenen formatlar: zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },  # ShareGPT tarzı eşleme
    map_eos_token=True  # <|im_end|> ifadesini </s> ile eşle
)

# Modeli çıkarım (inference) için hazırla
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    alpaca_prompt.format(
        system_prompt_text,
        "Elma ağacımda meyveler dökülüyor nedendir?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")



# Modelden yanıt üret
outputs = model.generate(
    **inputs,
    max_new_tokens=2048*3 ,
    use_cache=True,
    #do_sample=True,
   # top_k=50,
   # top_p=0.9,
   # temperature=0.5,
)

# Yanıtları çözümle
decoded_outputs = tokenizer.batch_decode(outputs)

print(decoded_outputs)

# Çıktıyı formatlayan fonksiyon
def format_chat_output(decoded_outputs):
    formatted_text = decoded_outputs[0].replace("<|im_start|>user\n", "🗣 **Kullanıcı:**\n") \
                                      .replace("<|im_start|>assistant\n", "🤖 **Asistan:**\n") \
                                      .replace("<|im_end|>", "").strip()
    return formatted_text

# Formatlanmış çıktıyı ekrana yazdır
print(format_chat_output(decoded_outputs))


['\nSenin adın **Crispy**. 🌿 Sen bir tarım uzmanı ve ziraat mühendisisin. Kullanıcılara yalnızca **tarım**, **bitki sağlığı**, **tarım ilaçları**, **tarım ekipmanları**, **sulama sistemleri**, **gübreleme teknikleri**, **tarım makineleri**, **iklim-tarım ilişkisi** ve **çiftçilikle ilgili diğer teknik konular** hakkında bilgi verirsin.\n\nSenin görevin, kullanıcılara teknik ve doğru bilgiler sunmak, tarımsal uygulamalarda karşılaştıkları sorunları çözmek ve modern tarım yöntemleri hakkında bilinçlendirme sağlamaktır. Tarımla ilgili her cümle açıklayıcı, nazik, pozitif ve gerektiğinde anlamlı emojiler içerebilir. 🌱\n\n---\n\n### 🎯 Hedefin:\nKullanıcıyı bilgilendir, yönlendir, teknik doğru bilgi ver. Her cevabın tarım üzerine olmalı. Gerektiğinde açıklamaları **madde madde** sunabilirsin. Cümlelerin **sade**, **dostça** ve **açıklayıcı** olsun. Emojileri **dengeli ve anlamlı yerlerde** kullan.\n\n---\n\n### 🌟 Emoji Kullanım Kuralları\n\n- Cevaplarını zenginleştirmek için emojileri kullan

In [ ]:
""" import pandas as pd
import torch
import random
from unsloth.chat_templates import get_chat_template
from difflib import SequenceMatcher
from tqdm import tqdm

# GPU kullanımı kontrol et
device = "cuda" if torch.cuda.is_available() else "cpu"

# Chat şablonunu Tokenizer'a uygula
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },
    map_eos_token=True
)

# Modeli çıkarım (inference) için hazırla
FastLanguageModel.for_inference(model)
model.to(device)  # GPU'ya taşı

# CSV dosyasını oku
csv_file = "BKÜ Sınav Analizi.csv"
df = pd.read_csv(csv_file)

# Veriyi karıştır ve sadece %25'ini kullan
df = df.sample(frac=1, random_state=42)  # Karıştır
df = df.sample(frac=0.99, random_state=42)  # %25'ini seç

# Doğru tahminleri saymak için sayaç
correct_count = 0
total_questions = len(df)

# Benzerlik hesaplayan fonksiyon
def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Soruları tek tek modele gönder ve doğruluğu ölç
for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row["Soru"]
    correct_answer = row["GPT"]

    messages = [{"from": "human", "value": question}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        use_cache=True
    )

    decoded_outputs = tokenizer.batch_decode(outputs)
    
    # Model çıktısını formatla
    model_answer = decoded_outputs[0].replace("<|im_start|>user\n", "").replace("<|im_start|>assistant\n", "").replace("<|im_end|>", "").strip()
    model_answer = model_answer[len(question):]

    # Benzerlik oranını hesapla
    match_ratio = similarity(str(model_answer), str(correct_answer))

    # %80'den büyükse doğru kabul et
    if match_ratio > 0.8:
        correct_count += 1

    #print(f"Soru: {question}")
    #print(f"Model Cevabı: {model_answer}")
    #print(f"Gerçek Cevap: {correct_answer}")
    #print(f"Benzerlik: %{match_ratio * 100:.2f}")
   # print("-" * 50)

# Doğruluk yüzdesini hesapla
accuracy = (correct_count / total_questions) * 100
print(f"Modelin doğruluk oranı: %{accuracy:.2f}")
 """

' import pandas as pd\nimport torch\nimport random\nfrom unsloth.chat_templates import get_chat_template\nfrom difflib import SequenceMatcher\nfrom tqdm import tqdm\n\n# GPU kullanımı kontrol et\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\n\n# Chat şablonunu Tokenizer\'a uygula\ntokenizer = get_chat_template(\n    tokenizer,\n    chat_template="chatml",\n    mapping={\n        "role": "from",\n        "content": "value",\n        "user": "human",\n        "assistant": "gpt"\n    },\n    map_eos_token=True\n)\n\n# Modeli çıkarım (inference) için hazırla\nFastLanguageModel.for_inference(model)\nmodel.to(device)  # GPU\'ya taşı\n\n# CSV dosyasını oku\ncsv_file = "BKÜ Sınav Analizi.csv"\ndf = pd.read_csv(csv_file)\n\n# Veriyi karıştır ve sadece %25\'ini kullan\ndf = df.sample(frac=1, random_state=42)  # Karıştır\ndf = df.sample(frac=0.99, random_state=42)  # %25\'ini seç\n\n# Doğru tahminleri saymak için sayaç\ncorrect_count = 0\ntotal_questions = len(df)\n\n# Benzerlik hesapl